# Genetic Application: Leukemia 

## Librairies

In [1]:
# !pip install deeplake
# !pip install -U scikit-learn

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
import os

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

import pickle

import pandas as pd
import numpy as np
import scipy.io

#import deeplake
from sklearn.datasets import fetch_openml

from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import KFold

from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest
from sklearn.feature_selection import f_classif, r_regression, chi2, mutual_info_classif

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA, SparsePCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score

from src.utils import *
from src.models import *

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [6]:
%load_ext autoreload
%autoreload 2

## Load data

In [7]:
# Load the Leukemia dataset from openml
leukemia_data = fetch_openml(name='Leukemia', version=1, as_frame=False, parser='auto')

data = leukemia_data.data
y = leukemia_data.target

URLError: <urlopen error [Errno 54] Connection reset by peer>

In [ ]:
data.shape, y.shape

## Parameters

In [50]:
nb_fts = 45    # 0.625%  pca = 45 fts ≠ 64 fts
# nb_fts = 89    # 1.25%   pca = 64 fts
# nb_fts = 178   # 2.50%   pca = 64 fts
# nb_fts = 356   # 5%      pca = 64 fts
# nb_fts = 712   # 10%     pca = 64 fts
### nb_fts = 1422  # 20%     pca = 64 fts

In [51]:
results_folder = f"results/genetic_data_LEUKEMIA/{nb_fts}" # separate folder for any nb_fts

if not os.path.exists(results_folder):
    os.mkdir(results_folder)

In [52]:
# Choose your models

models_l = ["knn", 
            "lr", 
            "svc", 
            "nb-gaussian", 
            ### "nb-bernouilli", 
            ### "nb-categorical",
            ### "rf"
           ]

# Choose your feature selection methods
fts_modes_l = [#"full", 
               #"random", 
               #"k-best",
               "k-best-mi", # neww 
               ###"pca", 
               # "sparse-pca", 
               ###"lfs", 
               ###"lbs", 
              ]

## Ten times 10-fold cross validation

In [53]:
def get_CV_splits(seed=42):

    cv_d = {"train_splits": [], "test_splits": []}

    kf = KFold(n_splits=10, shuffle=True, random_state=seed)
    kf.get_n_splits(data)

    for train_index, test_index in kf.split(data):

        cv_d["train_splits"].append(train_index)
        cv_d["test_splits"].append(test_index)
        
    return cv_d

In [54]:
cv_d = get_CV_splits(seed=42)
# cv_d

In [55]:
# 10 times 10-fold CV

cv_splits_all = []

for seed in tqdm([33, 42, 1, 5, 1979, 2024, 22, 12, 1996, 11]):
    
    cv_d = get_CV_splits(seed=seed)
    
    cv_splits_all.append(cv_d)

100%|███████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2223.80it/s]


In [56]:
# cv_splits_all

In [57]:
# *** new function ***
def select_features(train_indices, test_indices, data=data, y=y, norm=True, fts_mode="full"):
        
    # 2. fts selection
    if fts_mode == "random":
        rand_ind = np.random.randint(low=0, high=data.shape[1], size=nb_fts, dtype=int)
        current_data = data[:, rand_ind]
        
    else:
        current_data = data

    # 2. split
    # train set
    X_train_split = current_data[train_indices, :]
    if norm:
        X_train_split = normalize(X_train_split, axis=0)
    y_train_split = y[train_indices]

    label_encoder = LabelEncoder()
    y_train_split = label_encoder.fit_transform(y_train_split)
    y_train_split = 2 * y_train_split - 1

    # test set
    X_test_split = current_data[test_indices, :]
    if norm:
        X_test_split = normalize(X_test_split, axis=0)
    y_test_split = y[test_indices]
    y_test_split = label_encoder.transform(y_test_split)
    y_test_split = 2 * y_test_split - 1

    if fts_mode == "pca": # unsupervised
        pca = PCA(n_components=min(nb_fts, len(X_train_split))) # PCA limited by nb of rows of X (64)
        X_train_split = pca.fit_transform(X_train_split)
        X_test_split = pca.transform(X_test_split)

    if fts_mode == "sparse-pca": # unsupervised
        sparse_pca = SparsePCA(n_components=nb_fts, alpha=0.5, tol=1e-4, verbose=False)
        X_train_split = sparse_pca.fit_transform(X_train_split)
        X_test_split = sparse_pca.transform(X_test_split)

    if fts_mode == "lfs": # supervised
        # Note that the model used in the LFS algo and the downstream classifier (current_model) are the same!
        lfs = SequentialFeatureSelector(current_model, n_features_to_select=nb_fts, direction="forward")
        X_train_split = lfs.fit_transform(X_train_split, y_train_split)
        X_test_split = lfs.transform(X_test_split)

    if fts_mode == "lbs": # supervised
        # Note that the model used in the LFS algo and the downstream classifier (current_model) are the same!
        lfs = SequentialFeatureSelector(current_model, n_features_to_select=nb_fts, direction="backward")
        X_train_split = lfs.fit_transform(X_train_split, y_train_split)
        X_test_split = lfs.transform(X_test_split)

    if fts_mode == "k-best": # supervised
        # k_best = SelectKBest(chi2, k=nb_fts)
        k_best = SelectKBest(f_classif, k=nb_fts)
        X_train_split = k_best.fit_transform(X_train_split, y_train_split)
        X_test_split = k_best.transform(X_test_split) # no y here!

    if fts_mode == "k-best-mi": # supervised
        k_best = SelectKBest(mutual_info_classif, k=nb_fts)
        X_train_split = k_best.fit_transform(X_train_split, y_train_split)
        X_test_split = k_best.transform(X_test_split) # no y here!

    return X_train_split, y_train_split, X_test_split, y_test_split

In [58]:
# *** new function ***
def fit_model(X_train_split, y_train_split, X_test_split, y_test_split, model="knn"):

    # 1. model
    if model == "knn":
        current_model = KNeighborsClassifier()
    elif model == "lr":
        current_model = LogisticRegression()
    elif model == "svc":
        current_model = SVC()
    elif model == "nb-gaussian":
        current_model = GaussianNB()
    elif model == "nb-complement":
        current_model = ComplementNB()
    elif model == "nb-bernouilli":
        current_model = BernoulliNB()
    elif model == "nb-categorical":
        current_model = CategoricalNB()
    elif model == "rf":
        current_model = RandomForestClassifier()
    
    current_model.fit(X_train_split, y_train_split)
    y_test_preds = current_model.predict(X_test_split)

    # results
    # report = classification_report(y_test_split, y_test_preds)
    f1 = f1_score(y_test_split, y_test_preds, average='macro')
    b_acc = balanced_accuracy_score(y_test_split, y_test_preds)
        
    return f1, b_acc

## All experiments except Pk-LPNN at once

> - The following cell runs all feature selection modes (`fts_modes_l`) and all dowstream models (`models_l`).
> 
> - The results are then saved in `results_folder/`.
>
> - Hence, the individual sections (Full features, Random features, etc.) do not need to be executed anymore.

In [59]:
# *** new loop ***
# 10 times 10-fold CV: 100 experiments

results_all_d = {}

# 1. loop over feat modes:
for fts_mode in fts_modes_l:
        
    results_all_d[fts_mode] = {}

    # 2. 10 times 10-fold CV: 100 experiments
    for cv_d in tqdm(cv_splits_all):
        for train_indices, test_indices in zip(cv_d["train_splits"], cv_d["test_splits"]):
        
            X_train_split, y_train_split, X_test_split, y_test_split = select_features(train_indices, 
                                                                                       test_indices,
                                                                                       data=data, 
                                                                                       y=y, 
                                                                                       norm=True, 
                                                                                       fts_mode=fts_mode)
        
        
            # 3. loop over models
            for model in models_l:
    
                if model not in results_all_d[fts_mode].keys():
                    results_all_d[fts_mode][model] = {"f1" : [], "b_acc" : []}
                
                f1, b_acc = fit_model(X_train_split, y_train_split, X_test_split, y_test_split, model=model)
                
                results_all_d[fts_mode][model]["f1"].append(f1)
                results_all_d[fts_mode][model]["b_acc"].append(b_acc)

    # save all results for fts_mode
    for model in models_l:
        
        with open(os.path.join(results_folder, f"{fts_mode}_{nb_fts}_{model}.pkl"), "wb") as fh:
            pickle.dump(results_all_d[fts_mode][model], fh)

100%|████████████████████████████████████████████████████████████████████████| 10/10 [19:05<00:00, 114.55s/it]


In [60]:
for fts_mode in fts_modes_l:

    for model in models_l:
        print("*"*60)
        
        scores_full_fts = results_all_d[fts_mode][model]
    
        print(f"*** Features mode: {fts_mode} - Model: {model} ***")
        print(f"""Test: macro F1 (mean, std): \t\t{np.mean(scores_full_fts["f1"])}""")
        print(f"""Test: balanced accuracy (mean, std): \t{np.mean(scores_full_fts["b_acc"])}""")

************************************************************
*** Features mode: k-best-mi - Model: knn ***
Test: macro F1 (mean, std): 		0.9004923132423134
Test: balanced accuracy (mean, std): 	0.9330952380952382
************************************************************
*** Features mode: k-best-mi - Model: lr ***
Test: macro F1 (mean, std): 		0.8998256465756467
Test: balanced accuracy (mean, std): 	0.9330952380952382
************************************************************
*** Features mode: k-best-mi - Model: svc ***
Test: macro F1 (mean, std): 		0.31514613164613164
Test: balanced accuracy (mean, std): 	0.5257976190476191
************************************************************
*** Features mode: k-best-mi - Model: nb-gaussian ***
Test: macro F1 (mean, std): 		0.8914448051948053
Test: balanced accuracy (mean, std): 	0.9264642857142857


> STOP HERE!

## Pk-LPNN-selected features (normalized)

In [75]:
# Repeat 10 times:
#   10-fold CV
#   train PK-LPNN on 9 folds                     -> Nz selected fts
#   test PK-LPNN on 1 fold (KNN + selected fts)  -> b_acc, F1-score

In [76]:
def LPNN_experiment(p, Nz, k, mu_0=0.5, train_indices=None):

    # 1. split
    # train set
    X = data[train_indices, :]
    X = normalize(X, axis=0)

    label_encoder = LabelEncoder()
    Y = y[train_indices]
    Y = label_encoder.fit_transform(Y)
    Y = 2 * Y - 1

    # 2. Initialization
    beta_0, mu_0 = beta_0_and_mu_0(p=p, Nz=Nz, k=k, mu_0=mu_0, method="Pk-LPNN_v2")
    # check_conditions(beta, X, beta_0, n, Nz, k, method=method)

    # 3. dynamical system
    z0 = np.hstack([beta_0, mu_0])
    t_span = (0, 15) # (0, 300) # here
    t = t_span[1]
    eta = Nz

    # with tqdm() as pbar: # too much printing
        
    sol = solve_ivp(LPNN, 
                    t_span=t_span, 
                    y0=z0, 
                    args=(X, Y, eta, k, "Pk-LPNN_v2"), #, pbar), 
                    method="RK45", # DOP853, RK45
                    dense_output=False, 
                    max_step=0.1, 
                    atol=1.2e-4, 
                    rtol=1e-4)

    beta_sol = sol["y"][:-1, -1]
    mu_sol = sol["y"][-1, -1]

    selected_ind = np.argpartition(np.abs(beta_sol), -Nz)[-Nz:]
    
    return list(selected_ind)

In [77]:
# single experiment for selected features

def experiment(cv_d, data=data, y=y, norm=True, 
               train_indices=None, test_indices=None, selected_ind=None, 
               model="knn"):
    
    # 1. fts selection    
    current_data = data[:, selected_ind]

    # 2. split
    # train set
    X_train_split = current_data[train_indices, :]
    if norm:
        X_train_split = normalize(X_train_split, axis=0)
    y_train_split = y[train_indices]

    label_encoder = LabelEncoder()
    y_train_split = label_encoder.fit_transform(y_train_split)
    y_train_split = 2 * y_train_split - 1

    # test set
    X_test_split = current_data[test_indices, :]
    if norm:
        X_test_split = normalize(X_test_split, axis=0)
    y_test_split = y[test_indices]
    y_test_split = label_encoder.transform(y_test_split)
    y_test_split = 2 * y_test_split - 1

    # 3. model
    if model == "knn":
        current_model = KNeighborsClassifier()
    elif model == "lr":
        current_model = LogisticRegression()
    elif model == "svc":
        current_model = SVC()
    elif model == "nb-gaussian":
        current_model = GaussianNB()
    elif model == "nb-complement":
        current_model = ComplementNB()
    elif model == "nb-bernouilli":
        current_model = BernoulliNB()
    elif model == "nb-categorical":
        current_model = CategoricalNB()
    elif model == "rf":
        current_model = RandomForestClassifier()
    
    current_model.fit(X_train_split, y_train_split)
    y_test_preds = current_model.predict(X_test_split)

    # results
    # report = classification_report(y_test_split, y_test_preds)
    f1 = f1_score(y_test_split, y_test_preds, average='macro')
    b_acc = balanced_accuracy_score(y_test_split, y_test_preds)
    
    return f1, b_acc

In [78]:
# All experiments: 10 times 10-fold CV: 100 experiments

results_d = {}

for i, cv_d in tqdm(enumerate(cv_splits_all)):

    for train_indices, test_indices in zip(cv_d["train_splits"], cv_d["test_splits"]):
        # train set
        X_train_split = data[train_indices, :]
        y_train_split = y[train_indices]

        # parameters
        n = X_train_split.shape[0]
        p = X_train_split.shape[1]
        k = 1000
        # Nz_l = [int(X_train_split.shape[1] / 40)] # [100] # [20] # [150] # [100, 150, 200] # xxx useless now
        # eta_l = Nz_l                                                                       # xxx useless now
        # eta_l = eta_l[0]                                                                   # xxx useless now
        Nz = nb_fts # i.e. 178 Nz_l[0]
        # sigma = 0.02  # useless here, no noise
        mu_0 = 0.5

        # Pk-LPNN ft selection
        selected_ind = LPNN_experiment(p, Nz, k, mu_0=0.5, train_indices=train_indices)
        # print("selected_ind", selected_ind)
        
        # model with selected fts
        for model in models_l:

            if model not in results_d: # create dict if not exists
                results_d[model] = {"f1" : [], "b_acc" : []}
            
            f1, b_acc = experiment(cv_d, data=data, y=y, norm=True, 
                                   train_indices=train_indices, test_indices=test_indices, 
                                   selected_ind=selected_ind, model=model)
    
            results_d[model]["f1"].append(f1)
            results_d[model]["b_acc"].append(b_acc)

    print(f"CV {i+1} finished for all models.")
    

# save results
for model in models_l:
    
    with open(os.path.join(results_folder, f"pk-lpnn_{nb_fts}_{model}.pkl"), "wb") as fh: # xxx
        pickle.dump(results_d[model], fh)

    print(f"*** Features mode: Pk-LPNN - Model: {model} ***")
    print(f"""Test: macro F1 (mean, std): \t\t{np.mean(results_d[model]["f1"])}""")
    print(f"""Test: balanced accuracy (mean, std): \t{np.mean(results_d[model]["b_acc"])}""")

1it [05:38, 338.29s/it]

CV 1 finished for all models.


2it [10:54, 325.04s/it]

CV 2 finished for all models.


3it [17:22, 353.93s/it]

...

CV 8 finished for all models.


9it [57:07, 396.95s/it]

CV 9 finished for all models.


10it [1:04:19, 385.95s/it]

CV 10 finished for all models.
*** Features mode: Pk-LPNN - Model: knn ***
Test: macro F1 (mean, std): 		0.9148808968808969
Test: balanced accuracy (mean, std): 	0.9393452380952383
*** Features mode: Pk-LPNN - Model: lr ***
Test: macro F1 (mean, std): 		0.9108744033744033
Test: balanced accuracy (mean, std): 	0.9479166666666669
*** Features mode: Pk-LPNN - Model: svc ***
Test: macro F1 (mean, std): 		0.3251236541236541
Test: balanced accuracy (mean, std): 	0.5038333333333334
*** Features mode: Pk-LPNN - Model: nb-gaussian ***
Test: macro F1 (mean, std): 		0.842226134976135
Test: balanced accuracy (mean, std): 	0.8769404761904762
